### Heap-Hashmap Architecture

Note: since heap is not a stable sorting algorithm, we must order each element by both price and a timestamp. We also are not storing quantity in the heap, so the unique price and timestamp tuple must be some key in a hashmap storing the quantity available at that price. 

Currently I am not passing an order ID, nor an AuthorID for each order. In order to properly log the trades, we will need unique IDs for each order, each trade, and we will need to store all of those placing orders in another data structure, probably a hashmap, with clientID as the key and their actual alias as the value.

However, if we store only each available price level in the heap, we can maintain a priority queue associated with each price level, which will support linear amortized-time order book matching assuming there are an average of more than log(# of price levels) orders per price level.

In [11]:
#matching engine... should be queried before placing an order into the book. So it's not added if not necessary
#matching engine will use the current order, and check the book for orders that it could be paired to. If 
#pair-able, fill pair-able orders until no more (either fully filled, or partial fill with no more available at
#a pair-able price leve).

#for efficiency, we want to query all orders at a matchable price level, in price-time priority, such that we can
#just loop through those until done with the fill

#matching engines store the order book in memory.. so the book class should contain the me.. or really the ME 
#class should contain the book..

import heapq
import pandas as pd
from datetime import datetime

class matching_engine:
  def __init__(self):
    self.bids = []
    self.offers = []
    self.order_info = {}
    self.seq_num = 0
    self.seq_to_ID = {}

  def add_bid(self, price, qty, timestamp, order_ID, seq):
    if self.offers and self.match_bid(price, qty, timestamp, order_ID, seq):
      return
    heapq.heappush(self.bids, (-price, seq))
    self.seq_to_ID[seq] = order_ID
    self.order_info[order_ID] = [qty, seq, timestamp, -price]

  def add_offer(self, price, qty, timestamp, order_ID, seq):
    if self.bids and self.match_offer(price, qty, timestamp, order_ID, seq):
      return
    heapq.heappush(self.offers, (price, seq))
    self.seq_to_ID[seq] = order_ID
    self.order_info[order_ID] = [qty, seq, timestamp, price]

  def match_bid(self, price, qty, timestamp, order_ID, seq):
    min_offer = self.offers[0][0]
    min_offer_seq = self.offers[0][1]
    min_offer_ID = self.seq_to_ID[min_offer_seq]
    if price >= min_offer:
      min_offer_qty = self.order_info[min_offer_ID][0]
      trade_size = min(min_offer_qty, qty)
      qty -= trade_size
      self.order_info[min_offer_ID][0] -= trade_size

      print(f"Trade Executed: Bid ID = {order_ID}\tOffer ID = {min_offer_ID}size = {trade_size}\tprice = {min_offer}")

      if not self.order_info[min_offer_ID][0]:
        heapq.heappop(self.offers)
        del self.order_info[min_offer_ID]
        del self.seq_to_ID[min_offer_seq]
      if qty:
        self.add_bid(price, qty, timestamp, order_ID, seq)
      return True

    return False
    
  def match_offer(self, price, qty, timestamp, order_ID, seq):
    max_bid = -self.bids[0][0]
    max_bid_seq = self.bids[0][1]
    max_bid_ID = self.seq_to_ID[max_bid_seq]
    if price <= max_bid:
      max_bid_qty = self.order_info[max_bid_ID][0]
      trade_size = min(max_bid_qty, qty)
      qty -= trade_size
      self.order_info[max_bid_ID][0] -= trade_size

      print(f"Trade Executed: Bid ID = {max_bid_ID}\tOffer ID = {order_ID}\tsize = {trade_size}\tprice = {max_bid}")

      if not self.order_info[max_bid_ID][0]:
        heapq.heappop(self.bids)
        del self.order_info[max_bid_ID]
        del self.seq_to_ID[max_bid_seq]
      if qty:
        self.add_offer(price, qty, timestamp, order_ID, seq)
      return True

    return False
  
  def validate(self, o_type, price, qty, order_ID):
    valid = True
    if o_type not in 'bBoO':
      print(f"Error Log: Order Validation - Order Type Error: \nOrder [\'{o_type}\', {price}, {qty}, {order_ID}]\n\
Invalid order type: \'{o_type}\'.\nValid Orders are \'b\' or \'o\'.")
      valid = False
    if price <= 0:
      print(f"Error Log: Order Validation - Price Specification Error: \nOrder [\'{o_type}\', {price}, {qty}, {order_ID}]\n\
Invalid Price: {price}.\nValid Prices are Greater than Zero (p > 0)")
      valid = False
    if qty <= 0:
      print(f"Error Log: Order Validation - Size Specification Error: \nOrder [\'{o_type}\', {price}, {qty}, {order_ID}]\n\
Invalid Size: {price}.\nValid Sizes are Greater than Zero (qty > 0)")
      valid = False
    return valid

  def new_order(self, o_type, price, qty, order_ID):
    if not self.validate(o_type, price, qty, order_ID):
      return
    timestamp = datetime.now().timestamp()
    seq = self.seq_num
    self.seq_num += 1
    if o_type in 'bB':
      self.add_bid(price, qty, timestamp, order_ID, seq)
    else:
      self.add_offer(price, qty, timestamp, order_ID, seq)
    return

  def get_book(self):
    bids_copy = self.bids.copy()
    offers_copy = self.offers.copy()
    rows = []

    while bids_copy or offers_copy:
      cur_row = []
      if bids_copy:
        b_price, b_seq = heapq.heappop(bids_copy)
        b_ID = self.seq_to_ID[b_seq]
        b_qty = self.order_info[b_ID][0]
        cur_row.append(str(b_qty))
        cur_row.append(str(-b_price))
      else:
        cur_row.append('')
        cur_row.append('')
      
      if offers_copy:
        o_price, o_seq = heapq.heappop(offers_copy)
        o_ID = self.seq_to_ID[o_seq]
        o_qty = self.order_info[o_ID][0]
        cur_row.append(o_price)
        cur_row.append(o_qty)
      else:
        cur_row.append('')
        cur_row.append('')
      rows.append(cur_row)

    book = pd.DataFrame(columns=['bid_qty', 'bid_price', 'offer_price', 'offer_qty'], data=rows)

    display(book)
    return

ME = matching_engine()

ME.new_order('o', 12, 100, 'aa0000')
ME.new_order('b', 11.50, 50, 'aa0001')
ME.new_order('b', 11.75, 100, 'aa0002')
ME.new_order('b', 11.5, 25, 'aa0003')
ME.new_order('o', 12.05, 75, 'aa0004')
ME.new_order('o', 11.50, 80, 'aa0005')
ME.get_book()

Trade Executed: Bid ID = aa0002	Offer ID = aa0005	size = 80	price = 11.75


,bid_qty,bid_price,offer_price,offer_qty
0,20,11.75,12,100
1,50,11.5,12.05,75
2,25,11.5,,


### Heap-Hashmap-Priority Queue Architecture

by maintaining a priority queue to sort orders at the same price level, we don't even have to add timestamps. The priority queue is stable. In our heap, all we have to store is the same as the set of all keys in the hashmap. Each key corresponds to a price level that has at least an active order. Each unique order in the priority queue will have to contain a corresponding orderID, and quantity. Each element can be a len() 2 list containing (1) the order ID, and (2) the remaining quantity of that trade. All trades are assumed to be active until filled. Once we have a unique order ID for each order, we can store those IDs in their own DB (a map) to access them in linear time, allowing us to remove_order(order_ID) efficiently.

In [6]:
testheap = []
heapq.heappush(testheap, (10, '10'))
heapq.heappush(testheap, (10, '19'))
print(testheap)

[(10, '10'), (10, '19')]
